In [1]:
import os, argparse
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pytz import timezone
from pathlib import Path
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision
import wandb

import datasets
# from utils import select_device, natural_keys, gazeto3d, angular, getArch
from utils import select_device, natural_keys, gazeto3d, angular, getArch
from model import L2CS

In [2]:

args = argparse.Namespace()
args.gazeMpiimage_dir = '/project/data/sdata2/Image'  #real data 
args.gazeMpiilabel_dir = '/project/data/sdata2/Label'  #real label
# args.output = '/project/results/soutput1/snapshots/'
args.dataset = 'mpiigaze'
args.snapshot='/project/results/soutput2/snapshots/'
args.evalpath = '/project/results/sroutput2/evaluation/'
args.gpu_id = '0,1,2,3'
args.gpu_id = '0'
args.batch_size = 20
args.arch = 'ResNet50'
args.bins=35
args.angle = 180
args.bin_width = 4


In [3]:
batch_size=args.batch_size
arch=args.arch
data_set=args.dataset
evalpath =args.evalpath
snapshot_path = args.snapshot
bins=args.bins
angle=args.angle
bin_width=args.bin_width

In [4]:
# args = parse_args()
cudnn.enabled = True
gpu = select_device(args.gpu_id, batch_size=args.batch_size)
transformations = transforms.Compose([
    transforms.Resize(448),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

model_used=getArch(arch, bins)  #resnet50 and 28 bins

In [5]:
# check if we have the correct number of checkpoint files 
ppath = snapshot_path
for fold in range(15):
    foldstr = f"fold{fold:0>2}"
    cpath =os.path.join(ppath, foldstr)
    files = os.listdir(cpath)
    print(f'{fold}:{len(files)}',end=" ")

0:60 1:60 2:60 3:60 4:60 5:60 6:60 7:60 8:60 9:60 10:60 11:60 12:60 13:60 14:60 

In [6]:
def get_now():
    now = datetime.utcnow()
    now = now.astimezone(timezone('US/Pacific'))
    date_format='%m/%d/%Y %H:%M:%S'
    now = now.strftime(date_format) 
    return now
print(get_now())

07/12/2022 07:12:51


In [ ]:
%%time
all_MAE = []
# tosave={}
for fold in range(15):
    frun = wandb.init(project="62_syn_mode_sdata2_syn_data2")
    frun.name = f'fold{fold}_mae'
    
    print(f"fold={fold}")
    
    epoch_values=[]
    mae_values=[]

    now = get_now()
    
    print(args.gazeMpiilabel_dir)
    folder = os.listdir(args.gazeMpiilabel_dir)
    folder.sort()  #individual label files
    testlabelpathcombined = [os.path.join(args.gazeMpiilabel_dir, j) for j in folder] 
    
    gaze_dataset=datasets.Mpiigaze(testlabelpathcombined, args.gazeMpiimage_dir, transformations, False, angle, fold)

    test_loader = torch.utils.data.DataLoader(
        dataset=gaze_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True)

    fold_path = os.path.join(evalpath, 'fold' + f'{fold:0>2}'+'/')  #for evaluation
#     print(f"fold_path is {fold_path}")
    if not os.path.exists(fold_path):
        os.makedirs(fold_path)

#     if not os.path.exists(os.path.join(evalpath, f"fold"+str(fold))):
#         os.makedirs(os.path.join(evalpath, f"fold"+str(fold)))

    # list all epoch for testing
    folder = os.listdir(os.path.join(snapshot_path, "fold" + f'{fold:0>2}'))
    folder.sort(key=natural_keys)
#     folder.pop(-1)  #remove the tensorboard file, now all snapshot files
#     print(f"folder={folder}")
                    

    softmax = nn.Softmax(dim=1)
#     with open(os.path.join(evalpath, os.path.join("fold"+f'{fold:0>2}', data_set+".log")), 'w') as outfile:
        
    configuration = (f"\ntest configuration equal gpu_id={gpu}, batch_size={batch_size}, model_arch={arch}\n"
                     f"Start testing dataset={data_set}, FOLD={fold} --{now}---------")
    print(configuration)
#     tosave['time':now]
#     outfile.write(configuration)
    epoch_list=[]
    avg_MAE=[]
    for epoch in folder: 
        print(f"entering epoch={epoch}")
        model=model_used
        checkpoint = torch.load(os.path.join(snapshot_path+"fold"+f'{fold:0>2}', epoch))
        saved_state_dict = checkpoint['model_state_dict']
        model= nn.DataParallel(model,device_ids=[0])
        model.load_state_dict(saved_state_dict)
        model.cuda(gpu)
        model.eval()
        total = 0
        idx_tensor = [idx for idx in range(35)]
        idx_tensor = torch.FloatTensor(idx_tensor).cuda(gpu)
        avg_error = .0
        with torch.no_grad():
            for j, (images, labels, cont_labels, name) in enumerate(test_loader):
                images = Variable(images).cuda(gpu)
                total += cont_labels.size(0)

                label_pitch = cont_labels[:,0].float()*np.pi/180
                label_yaw = cont_labels[:,1].float()*np.pi/180

                gaze_pitch, gaze_yaw = model(images)

                # Binned predictions
                _, pitch_bpred = torch.max(gaze_pitch.data, 1)
                _, yaw_bpred = torch.max(gaze_yaw.data, 1)

                # Continuous predictions
                pitch_predicted = softmax(gaze_pitch)
                yaw_predicted = softmax(gaze_yaw)

                # mapping from binned (0 to 28) to angels (-42 to 42)                
                pitch_predicted = \
                    torch.sum(pitch_predicted * idx_tensor, 1).cpu() * 3 - 52
                yaw_predicted = \
                    torch.sum(yaw_predicted * idx_tensor, 1).cpu() * 3 - 52

                pitch_predicted = pitch_predicted*np.pi/180
                yaw_predicted = yaw_predicted*np.pi/180

                for p,y,pl,yl in zip(pitch_predicted, yaw_predicted, label_pitch, label_yaw):
#                     pl, yl = yl, pl*(-1.0)
#                     yl = yl*(-1.0)
                    avg_error += angular(gazeto3d([p,y]), gazeto3d([pl,yl]))

        x = ''.join(filter(lambda i: i.isdigit(), epoch))
#             print(f"x={x}")
        
        epoch_list.append(x)
        mean_mae = avg_error/total  #mean mae over the 3000 iamges
        avg_MAE.append(mean_mae)  
#             print(f"total={total}")
        now = get_now()
        loger = f"[{epoch}---{args.dataset}] Total Num:{total},MAE:{mean_mae}  {now}"
#         outfile.write(loger)
        print(loger)
#             print(f"done epoch={epochs}")
        epochn = int(x)
#         wandb.log({'epoch':epochn,'total':total, 'MAE':mean_mae, 'FOLD':fold })
#         tosave ={'epoch':epochn,'total':total, 'MAE':mean_mae, 'FOLD':fold }
        epoch_values.append(epochn)
        mae_values.append(mean_mae)
# #         wandb.log(tosave)
        wandb.log({'epoch': epoch, f'fold_{fold}_mean_mae':mean_mae}, step=epochn)
       
    all_MAE.append(avg_MAE)
    wandb.finish()


wandb: Currently logged in as: synthesis-ai (use `wandb login --relogin` to force relogin)


fold=0
/project/data/sdata2/Label
32 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=0 --07/12/2022 07:12:55---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:2967,MAE:5.465974983706747  07/12/2022 07:13:18
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:2967,MAE:3.4218344218957824  07/12/2022 07:13:32
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:2967,MAE:3.7666034222228295  07/12/2022 07:13:47
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:2967,MAE:3.4657238103488512  07/12/2022 07:14:01
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:2967,MAE:3.837612394686458  07/12/2022 07:14:16
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:2967,MAE:3.850513022970067  07/12/2022 07:14:30
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:2967,MAE:3.2062493564091943  07/12/2022 

fold_0_mean_mae,█▂▃▄▂▅▃▃▃▃▂▃▃▁▃▁▃▁▂▁▁▂▁▂▂▂▂▂▁▃▃▁▂▃▂▂▃▂▂▂
epoch,epoch_60.pkl
fold_0_mean_mae,3.27558


fold=1
/project/data/sdata2/Label
29 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=1 --07/12/2022 07:27:27---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:2970,MAE:4.234929974829739  07/12/2022 07:27:44
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:2970,MAE:3.922033376024581  07/12/2022 07:27:58
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:2970,MAE:4.325969221146043  07/12/2022 07:28:13
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:2970,MAE:2.982599724697332  07/12/2022 07:28:27
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:2970,MAE:2.838662498962718  07/12/2022 07:28:41
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:2970,MAE:3.2234165486650106  07/12/2022 07:28:55
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:2970,MAE:3.128414365812011  07/12/2022 07:

fold_1_mean_mae,█▇▂▂▃▅▂▄▃▂▄▃▂▃▃▂▁▂▁▂▂▂▂▁▃▁▂▂▁▁▂▁▂▂▁▁▁▁▁▂
epoch,epoch_60.pkl
fold_1_mean_mae,2.91831


...

fold=9
/project/data/sdata2/Label
46 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=9 --07/12/2022 09:26:07---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:2953,MAE:6.26316252400032  07/12/2022 09:26:24
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:2953,MAE:5.705340637967565  07/12/2022 09:26:38
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:2953,MAE:5.553179142097324  07/12/2022 09:26:54
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:2953,MAE:5.418368349965167  07/12/2022 09:27:09
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:2953,MAE:5.859532581547459  07/12/2022 09:27:24
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:2953,MAE:5.3558157226567324  07/12/2022 09:27:38
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:2953,MAE:5.501137355048879  07/12/2022 09:2

fold_9_mean_mae,█▅▄▆▄▃▃▄▃▄▄▃▂▃▂▃▄▂▃▃▄▁▂▂▂▂▂▂▂▂▃▃▃▃▃▂▂▂▂▃
epoch,epoch_60.pkl
fold_9_mean_mae,5.10194


fold=10
/project/data/sdata2/Label
37 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=10 --07/12/2022 09:40:45---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:2962,MAE:4.896690534683522  07/12/2022 09:41:02
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:2962,MAE:4.826738656898449  07/12/2022 09:41:16
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:2962,MAE:3.6787379232706754  07/12/2022 09:41:30
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:2962,MAE:3.9370182507483777  07/12/2022 09:41:45
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:2962,MAE:4.251568481812761  07/12/2022 09:41:59
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:2962,MAE:3.608697262298723  07/12/2022 09:42:14
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:2962,MAE:3.6044349293150217  07/12/2022

fold_10_mean_mae,██▄▆▃▃▃▂▂▄▁▁▂▂▂▂▃▁▂▂▂▂▁▃▂▂▂▂▁▂▁▂▂▂▂▂▃▁▂▂
epoch,epoch_60.pkl
fold_10_mean_mae,3.32756


fold=11
/project/data/sdata2/Label
36 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=11 --07/12/2022 09:55:33---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:2963,MAE:5.805708633662411  07/12/2022 09:55:50
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:2963,MAE:4.515521929859549  07/12/2022 09:56:05
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:2963,MAE:5.5398847158410955  07/12/2022 09:56:19
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:2963,MAE:3.1422142460379936  07/12/2022 09:56:33
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:2963,MAE:3.6788063547075667  07/12/2022 09:56:47
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:2963,MAE:3.5744927076469737  07/12/2022 09:57:03
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:2963,MAE:2.829784653629727  07/12/202

In [14]:
all_MAE=np.array(all_MAE)

In [1]:

print(all_MAE.shape)
print(all_MAE.mean(axis=0))
print(all_MAE.mean(axis=1))
print(all_MAE.mean(axis=0).argmin()+1 ,all_MAE.mean(axis=0).min())

NameError: name 'all_MAE' is not defined

In [ ]:
best_epoch = all_MAE.mean(axis=0).argmin()+1
min_error = all_MAE.mean(axis=0).min()
mean_errors = all_MAE.mean(axis=0)
print(best_epoch,min_error, mean_errors.shape )

In [ ]:
xx = mean_errors
min_error = 100
with wandb.init(project="62_syn_mode_sdata2_syn_data2") as run:
    run.name = "mean_error_per_epoch"
    for epoch in range(len(xx)):
        err = xx[epoch]
        if err < min_error:
            min_error = err
            run.summary["minimum_error (degree)"] = min_error
        run.log({f'mean epoch error (degree)':err}, step=epoch)
        

In [ ]:
save_file = Path(snapshot_path).parent/"syn_syn_sdata2_all_MAE.npy"
with open(save_file, 'wb') as f:
    np.save(f, all_MAE)

In [ ]:
for idx, fdata in enumerate(all_MAE):
    print(idx, np.argmin(fdata)+1, fdata.min())